In [3]:
import cupy
import subprocess
import matplotlib.pyplot as plt
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D
import time
import line_profiler as lp
from cupyx.profiler import benchmark
from numba import cuda
show_hardware_info = 0
if show_hardware_info:
    cpu_info = subprocess.run('lscpu');
    gpu_info = subprocess.run("nvidia-smi -L".split());

def build_up_b(b, rho, dt, u, v, dx, dy):
    b[1:-1, 1:-1] = (rho * (1 / dt * 
                    ((u[1:-1, 2:] - u[1:-1, 0:-2]) / 
                     (2 * dx) + (v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy)) -
                    ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx))**2 -
                      2 * ((u[2:, 1:-1] - u[0:-2, 1:-1]) / (2 * dy) *
                           (v[1:-1, 2:] - v[1:-1, 0:-2]) / (2 * dx))-
                          ((v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy))**2))
    return b
@cuda.jit
def pressure_poisson(p, dx, dy, b):
    i,j = cuda.grid(2)
    
    if i < p.shape[0] and j < p.shape[1]:
        for q in range(nit):
            tmp = 0.
            tmp += ((p[i+1,j]+p[i-1,j]) * dy**2 +(p[i,j+1]+p[i,j-1]) * dx**2)/(2 * (dx**2 + dy**2)) - dx**2 *dy**2/ (2*(dx**2+dy**2))*b[i,j]
            p[i,j] = tmp
        cuda.syncthreads()
def cavity_flow(nt, u, v, dt, dx, dy, p, rho, nu):
    un = np.zeros_like(u)
    vn = np.zeros_like(v)
    b = np.zeros_like(p)
    
    for n in range(nt):
        #When we reach this statement, the latest timestep is saved in u and v
        #We could copy this data to un and vn, but here we just swap pointers to save a copy operation
        u,un=un,u
        v,vn=vn,v
        
        b = build_up_b(b, rho, dt, un, vn, dx, dy)
        threadsperblock = (16,16)
        blockspergrid_x = int(np.ceil(p.shape[0]/threadsperblock[0]))
        blockspergrid_y = int(np.ceil(p.shape[1]/threadsperblock[1]))
        blockspergrid = (blockspergrid_x,blockspergrid_y)
        pressure_poisson[blockspergrid,threadsperblock](p, dx, dy, b)        
        u[1:-1, 1:-1] = (un[1:-1, 1:-1]-
                         un[1:-1, 1:-1] * dt / dx *
                        (un[1:-1, 1:-1] - un[1:-1, 0:-2]) -
                         vn[1:-1, 1:-1] * dt / dy *
                        (un[1:-1, 1:-1] - un[0:-2, 1:-1]) -
                         dt / (2 * rho * dx) * (p[1:-1, 2:] - p[1:-1, 0:-2]) +
                         nu * (dt / dx**2 *
                        (un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, 0:-2]) +
                         dt / dy**2 *
                        (un[2:, 1:-1] - 2 * un[1:-1, 1:-1] + un[0:-2, 1:-1])))

        v[1:-1,1:-1] = (vn[1:-1, 1:-1] -
                        un[1:-1, 1:-1] * dt / dx *
                       (vn[1:-1, 1:-1] - vn[1:-1, 0:-2]) -
                        vn[1:-1, 1:-1] * dt / dy *
                       (vn[1:-1, 1:-1] - vn[0:-2, 1:-1]) -
                        dt / (2 * rho * dy) * (p[2:, 1:-1] - p[0:-2, 1:-1]) +
                        nu * (dt / dx**2 *
                       (vn[1:-1, 2:] - 2 * vn[1:-1, 1:-1] + vn[1:-1, 0:-2]) +
                        dt / dy**2 *
                       (vn[2:, 1:-1] - 2 * vn[1:-1, 1:-1] + vn[0:-2, 1:-1])))
        
        u[0, :]  = 0
        u[:, 0]  = 0
        u[:, -1] = 0
        u[-1, :] = 1    # set velocity on cavity lid equal to 1
        v[0, :]  = 0
        v[-1, :] = 0
        v[:, 0]  = 0
        v[:, -1] = 0
        
        
    return u, v, p

#Run performance measurement for set of matrix dimensions (nx_list) and alternating CPU/GPU (run_on_gpu_list)

#Table headings for profiling output data
print("#run_on_gpu [bool]\tnx [-]\ttime [s]")

#List of matrix dimensions to test
#For GPU: fist run seems to have an extra delay. Run small case in the beginning to prevent this from affecting the profiling results
#nx_list = [128, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]
nx_list = [128, 128, 256, 512]

#Run on CPU and/or GPU
run_on_gpu_list = [1,1] #Run on both CPU and GPU
#run_on_gpu_list = [0] #Run only on CPU
#run_on_gpu_list = [1] #Run only on GPU

for nx in nx_list:
    for run_on_gpu in run_on_gpu_list:
        if run_on_gpu:
            import cupy as np
        else:
            import numpy as np
            
        ny = nx
        nt = 10 #NB! Value lowered from original code so that profiling runs don't take too much time
        nit = 10 #NB! Inaccurate with only 10! Same comment as line above
        c = 1
        dx = 2 / (nx - 1)
        dy = 2 / (ny - 1)
        
        rho = 1
        nu = .1
        dt = .000001 #Needs to be very small to be stable with a very fine grid
        
        plot_results = 0 #[bool] Set to 0 for profiling, maybe 1 for debugging
        u = np.zeros((ny, nx), dtype=np.float32) #NB! Change here to get float or double
        v = np.zeros_like(u)
        p = np.zeros_like(u)
        
        #Code for actual profiling.
        #The cavity_flow() call together with the synchronize() calls take about the same time as benchmark()
        #Both profiling methods are probably fine
        cupy.cuda.stream.get_current_stream().synchronize()
        start_iterative = time.time()
        #u, v, p = cavity_flow(nt, u, v, dt, dx, dy, p, rho, nu)
        benchmark_result = benchmark(cavity_flow, (nt, u, v, dt, dx, dy, p, rho, nu,), n_warmup=0, n_repeat=1)
        #print(benchmark_result)
        cupy.cuda.stream.get_current_stream().synchronize()
        end_iterative = time.time()
        
        #print line for table of profiling results
        print(f'{run_on_gpu}\t\t\t{nx}\t{end_iterative - start_iterative}')
        
        if plot_results:
            x = np.linspace(0, 2, nx)
            y = np.linspace(0, 2, ny)
            X, Y = np.meshgrid(x, y)
            if run_on_gpu:
                X = X.get()
                Y = Y.get()
                p = p.get()
                u = u.get()
                v = v.get()
            
            fig = pyplot.figure(figsize=(11,7), dpi=100)
            # plotting the pressure field as a contour
            pyplot.contourf(X, Y, p, alpha=0.5, cmap=cm.viridis)  
            pyplot.colorbar()
            # plotting the pressure field outlines
            pyplot.contour(X, Y, p, cmap=cm.viridis)  
            # plotting velocity field
            pyplot.quiver(X[::2, ::2], Y[::2, ::2], u[::2, ::2], v[::2, ::2]) 
            pyplot.xlabel('X')
            pyplot.ylabel('Y');
            pyplot.show()




#run_on_gpu [bool]	nx [-]	time [s]
1			128	0.19191598892211914
1			128	0.02517247200012207
1			128	0.024414539337158203
1			128	0.023488283157348633
1			256	0.02222418785095215
1			256	0.022118330001831055
1			512	0.033768653869628906
1			512	0.03352832794189453
